In [1]:
import numpy as np
import os
import pandas as pd
import glob
import h5py
from ast import literal_eval
import pandas_profiling

In [2]:
#### Path to all data files

# All weather
weather17 = ["./Data/GFS_2017/" + f for f in os.listdir("./Data/GFS_2017") if f.startswith('gfs')]
weather18 = ["./Data/GFS_2018/" + f for f in os.listdir("./Data/GFS_2018") if f.startswith('gfs')]
weather19 = ["./Data/GFS_2019/" + f for f in os.listdir("./Data/GFS_2019") if f.startswith('gfs')]
allweather = weather17 + weather18 + weather19

# All impact
allimpact = ["./Data/" + f for f in os.listdir("./Data") if f.startswith('impact')]

# Loc2zip map
grid = "./Data/search_grid.csv"

In [3]:
weather = [pd.read_csv(f, engine="python") for f in allweather]

In [ ]:
weather = pd.concat(map(pd.read_csv, allweather), ignore_index=True)

In [ ]:
### Load and flatten loc2zip map
loc2zip = pd.read_csv(grid, converters={"mapped_zipcodes": literal_eval})
loc2zip = loc2zip.explode("mapped_zipcodes").reset_index()
loc2zip['mapped_zipcodes'] = pd.to_numeric(loc2zip['mapped_zipcodes'])
#loc2zip.rename(columns={"mapped_zipcodes":"zip5"}, inplace=True)

In [ ]:
### Load impact into 1 df
impact = pd.concat(map(pd.read_csv, allimpact), ignore_index=True).drop(["Unnamed: 0"], axis=1)

In [ ]:
### Join impact and loc2zip
impact_loc = impact.join(loc2zip.set_index('mapped_zipcodes'), on='zip5').drop(["index"], axis = 1)

In [ ]:
### Join impact and weather in loc and date
# Preprocessing on join columns 
impact_loc["date_key"] = pd.to_datetime(impact_loc["date_key"])
weather["Date"] = pd.to_datetime(weather["Date"], format="%Y%m%d")

data = impact_loc.merge(weather, left_on=["date_key", "grid_lat", "grid_lon"], right_on=["Date", "lat", "lng"])
data = data.sort_values(by="Date")

In [ ]:
nan_cols = [i for i in data.columns if data[i].isnull().any()]
df = data[nan_cols]

In [ ]:
profile = pandas_profiling.ProfileReport(data, title='Pandas Profiling Report', html={'style':{'full_width':True}})

In [ ]:
profile2 = pandas_profiling.ProfileReport(df, title='Pandas Profiling Report 2', html={'style':{'full_width':True}}, minimal=True)

In [ ]:
profile.to_file(output_file="full.html")

In [ ]:
profile2.to_file(output_file="nans.html")

In [ ]:
data = data.sort_values(["zip5", 'date_key', 'Time'])
x = data['zip5']

In [ ]:
far = pd.read_excel('FARcodesZIPdata2010WithAKandHI.xlsx', sheet_name = 1)

In [ ]:
cols = ['ZIP', 'density', 'state']
far = far[cols]
new_row = {'ZIP': 2722, 'density': 0, 'state': 'MA'}
far = far.append(new_row, ignore_index=True)

In [ ]:
data = data.merge(far, left_on = 'zip5', right_on = 'ZIP')
y = data['zip5']

In [ ]:
New_England_Northeast = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
Mid_Atlantic_Northeast = ['NJ', 'NY', 'PA']
East_North_Central_Midwest = ['IL', 'IN', 'MI', 'OH', 'WI'] 
West_North_Central_Midwest = ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']
South_Atlantic_South = ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'DC', 'WV']
East_South_Central_South = ['AL', 'KY', 'MS', 'TN']
West_South_Central_South = ['AR', 'LA', 'OK', 'TX']
Mountain_West = ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY']
Pacific_West = ['AK', 'CA', 'HI', 'OR', 'WA']

In [ ]:
data['USR'] = ['Urban' if x > 3000 else 'Rural' if x < 1000 else 'Suburban' for x in data['density']]

In [ ]:
data['Region'] = ['New England Northeast' if x in New_England_Northeast else
                 'Mid Atlantic Northeast' if x in Mid_Atlantic_Northeast else
                 'East North Central Midwest' if x in East_North_Central_Midwest else
                 'West North Central Midwest' if x in West_North_Central_Midwest else
                 'South Atlantic South' if x in South_Atlantic_South else
                 'East South Central South' if x in East_South_Central_South else
                 'West South Central South' if x in West_South_Central_South else
                 'Mountain West' if x in Mountain_West else
                 'Pacific West' for x in data['state']]
data = data.drop(["ZIP"], axis=1)

In [ ]:
data['Weekday'] = data['date_key'].dt.dayofweek

In [ ]:
data